In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from keras.preprocessing import image

In [4]:
tf.test.is_built_with_cuda()

In [5]:
#to see GPU name on system
tf.config.experimental.list_physical_devices()

In [6]:
traindatapath='../input/11-785-s22-hw2p2-classification/classification/classification/train'
devdatapath='../input/11-785-s22-hw2p2-classification/classification/classification/dev'
testdatapath='../input/11-785-s22-hw2p2-classification/classification/classification/test'

In [7]:
#Extracting and Scaling Training Images
train_gen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_gen.flow_from_directory(traindatapath,target_size=(64,64),batch_size=100,class_mode='categorical')

In [8]:
#Extracting and Scaling Validation Images
val_gen = ImageDataGenerator(rescale=1.0/255)
val_generator = val_gen.flow_from_directory(devdatapath,target_size=(64,64),batch_size=100,class_mode='categorical')

In [9]:
len(np.unique(train_generator.labels))

In [ ]:
    #Model Building
    base_model = VGG16(input_shape = (64,64,3),include_top = False,weights = 'imagenet',classes = 7000)
    for layer in base_model.layers:
        layer.trainable = False
    model= Sequential()
    model.add(base_model) 
    model.add(Flatten())
    model.add(Dense(1024,activation=('relu')))
    model.add(Dense(512,activation=('relu')))
    model.add(Dense(256,activation=('relu'))) 
    model.add(Dense(7000,activation=('softmax')))
    adam = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=adam,loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

In [ ]:
#Model Fitting
with tf.device('/GPU:0'):
    history = model.fit_generator(train_generator,epochs=50,steps_per_epoch=1400,validation_data=val_generator,validation_steps=350)

In [ ]:
#Saving Trained Model
model.save("minimodel.h5")

In [11]:
#Loading Trained Model
modelpath='./minimodel.h5'
model = tf.keras.models.load_model(modelpath)

In [44]:
#Evaluating Model on Training images
with tf.device('/GPU:0'):
    model.evaluate(train_generator)

In [74]:
#Evaluating Model on Validation Images
with tf.device('/GPU:0'):
    model.evaluate(val_generator)

In [14]:
#Extracting Test Images
with tf.device('/GPU:0'):
    testimagedirs = os.listdir(testdatapath)
    test=[]
    for image_name in testimagedirs:
        imagepath = testdatapath + "/" + image_name
        img = image.load_img(imagepath)
        imgg = img.resize([64, 64])
        arr=np.array(imgg)
        arr=arr.flatten()
        test.append(arr)
        print('loading img..',image_name)
    print('done')

In [15]:
testimages=np.array(test)

In [16]:
#Scaling Test Images
testimages=testimages/255.0

In [17]:
testimages.shape

In [18]:
#Reshaping to display images
testimages = testimages.reshape(35000, 64, 64, 3)

In [19]:
plt.matshow(testimages[0])

In [20]:
len(testimages)

In [21]:
#Predicting labels
with tf.device('/GPU:0'):   
    pred=model.predict(testimages)
    predlabs = np.argmax(pred, axis=1)

In [22]:
predlabs

In [23]:
#DataFrame with final labels
resdf=pd.DataFrame(data=predlabs)

In [24]:
#Final csv to submit on Kaggle
resdf.to_csv('sub.csv')